In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import seaborn as sns

from pymongo import MongoClient
mc = MongoClient(os.getenv('mongo_host'),
                    username=os.getenv('mongo_user'),
                    password=os.getenv('mongo_pass'),
                    authSource=os.getenv('mongo_db_auth'),
                    authMechanism='SCRAM-SHA-256')

db = mc['bfv_processing']

In [7]:
# Data
df = pd.DataFrame(db.processed_match_players.find({"match_start_time":{"$gte":datetime(2021, 9, 1)}, "team_status":{"$ne":"dnf"}}))
df['week'] = df['match_start_time'].dt.to_period('W').apply(lambda r: r.start_time)
df['day'] = df['match_start_time'].dt.to_period('d').apply(lambda r: r.start_time)

In [8]:
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pio.templates.default = "none"

# Seaborn Config
p1 = sns.color_palette("dark:#0B6872", n_colors=10)
p2 = sns.color_palette("dark:#00D3BC", n_colors=10)
p3 = sns.color_palette("light:#040404", n_colors=10)
p4 = sns.color_palette("dark:#F4E9C3", n_colors=10)
p5 = sns.color_palette("dark:#EF8B79", n_colors=10)

# Colors
gumi_palette = ['#0B6872','#00D3BC','#040404','#F4E9C3','#EF8B79']

gumi_layout_update_dict = dict(
    font_family="sans-serif",
    font_color="grey",
    title_font_family="sans-serif",
    title_font_size=20,
    title_font_color="black",
    legend_title_font_color="grey",
    showlegend=False,
    paper_bgcolor='#E9F2F1',
    plot_bgcolor='rgba(0,0,0,0)'
)

In [15]:
def gen_gumi_metric_breakdown(df, metric):
    """
    Breaks down a metric
    """

    # Printing Parameters
    fname = ' '.join([x.capitalize() for x in metric.split('_')])

    # Compute setup
    gumi_squad = ['psn_GotYourBach', 'psn_MissNemisis', 'psn_SloshySole', 'psn_mypremeclean', 'psn_drumcon11']
    gumi = df[df.player_id.isin(gumi_squad)]
    last_week = gumi[gumi.week > datetime.now() - timedelta(days=7)]
    last_2_weeks = gumi[gumi.week > datetime.now() - timedelta(days=14)]
    mean_performance = last_week[['player_id', metric]].groupby(['player_id'], as_index=False).mean().sort_values(metric, ascending=False).reset_index()
    peak_performance = last_week[['player_id', metric]].groupby(['player_id'], as_index=False).max().sort_values(metric, ascending=False).reset_index()

    # Percentiles
    top_1 = df[metric].quantile(.99).round(1)
    top_5 = df[metric].quantile(.95).round(1)
    top_10 = df[metric].quantile(.90).round(1)

    # Printing
    print(f"Metric: {fname}")
    print('{0:<15}'.format(f'Top 1% {top_1} | ') + '{0:<15}'.format(f'Top 5% {top_5} | ') + '{0:<15}'.format(f'Top 10% {top_10} | '))
    print("Current Leaderboard: ")
    print("{0:^24}".format('Average Performance') + ' | ' + "{0:^24}".format('Peak Performance'))
    print("="*50)
    for index, row in mean_performance.iterrows():
        print(str(index+1) + " | " + 
        "{0:<16}".format(row['player_id'])[4:] + " | " + 
        "{0:^5}".format(round(row[metric], 2)) + " | " +
        str(index+1) + " | " + 
        "{0:<16}".format(peak_performance.loc[index, 'player_id'])[4:] + " | " + 
        "{0:^5}".format(round(peak_performance.loc[index, metric], 2)))

    # Weekly Line chart
    g1 = gumi[['player_id', 'week', metric]].groupby(['week', 'player_id'], as_index=False).mean()
    fig1 = px.line(g1, x='week', y=metric, color='player_id', color_discrete_sequence=gumi_palette, text=g1[metric].round(2))
    fig1.update_traces(textposition="bottom right", line=dict(width=4))


    # Daily
    g2 = last_2_weeks[['player_id', 'day', metric]].groupby(['day', 'player_id'], as_index=False).mean()
    fig2 = px.line(g2, x='day', y=metric, color='player_id', color_discrete_sequence=gumi_palette, text=g2[metric].round(2))
    fig2.update_traces(textposition="bottom right", line=dict(width=4))
   
    # Plot
    out = make_subplots(rows=1, cols=2)
    out.update_layout(height=600, width=2000, title_text=f"<b>{fname} Performance by Player, Last 60/21 Days</b>", shapes=[dict(type='line',
                x0=g1['week'].min(),
                y0=top_10,
                x1=g1['week'].max(),
                y1=top_10,
                line=dict(color='#00B551', dash='dash'),
                xref='x',
                yref='y',
                opacity=.4
)], **gumi_layout_update_dict)

    for trace in fig1['data']:
        out.append_trace(trace, row=1, col=1)

    for trace in fig2['data']:
        out.append_trace(trace, row=1, col=2)


    out.show()


In [16]:
metrics = ['AER', 'aggression_rating', 'efficiency_rating', 'score_per_min', 
'kills_per_death', 'true_kills_per_death', 'kills_per_min', 'solider_damages',
 'shots_taken', 'shots_hit', 'shot_accuracy',
 'heals', 'revives', 'revives_recieved', 'resupplies', 'repairs', 'squad_spawns', 'squad_wipes','orders_completed', 
'overall_rank', 'team_rank', 'player_time']
for metric in metrics:
    gen_gumi_metric_breakdown(df, metric)

Metric: Aer
Top 1% 9.5 |   Top 5% 8.1 |   Top 10% 7.2 |  
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | GotYourBach  | 7.82  | 1 | GotYourBach  | 10.0 
2 | MissNemisis  | 7.43  | 2 | MissNemisis  | 10.0 
3 | SloshySole   | 7.15  | 3 | SloshySole   | 9.56 
4 | mypremeclean | 5.98  | 4 | mypremeclean | 9.24 
5 | drumcon11    | 3.64  | 5 | drumcon11    | 5.47 


Metric: Aggression Rating
Top 1% 5.0 |   Top 5% 3.7 |   Top 10% 3.2 |  
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | GotYourBach  | 3.56  | 1 | GotYourBach  |  5.0 
2 | SloshySole   | 3.47  | 2 | MissNemisis  |  5.0 
3 | MissNemisis  | 3.45  | 3 | SloshySole   |  5.0 
4 | mypremeclean |  2.6  | 4 | mypremeclean | 4.24 
5 | drumcon11    | 1.77  | 5 | drumcon11    | 2.43 


Metric: Efficiency Rating
Top 1% 5.0 |   Top 5% 5.0 |   Top 10% 5.0 |  
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | GotYourBach  | 4.26  | 1 | GotYourBach  |  5.0 
2 | MissNemisis  | 3.98  | 2 | MissNemisis  |  5.0 
3 | SloshySole   | 3.68  | 3 | SloshySole   |  5.0 
4 | mypremeclean | 3.38  | 4 | mypremeclean |  5.0 
5 | drumcon11    | 1.86  | 5 | drumcon11    | 3.04 


Metric: Score Per Min
Top 1% 800.5 | Top 5% 597.0 | Top 10% 512.2 | 
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | GotYourBach  | 639.0 | 1 | SloshySole   | 1360.0
2 | SloshySole   | 613.21 | 2 | GotYourBach  | 1182.0
3 | MissNemisis  | 597.09 | 3 | MissNemisis  | 953.62
4 | mypremeclean | 438.18 | 4 | mypremeclean | 721.0
5 | drumcon11    | 324.38 | 5 | drumcon11    | 431.75


Metric: Kills Per Death
Top 1% 13.0 |  Top 5% 4.8 |   Top 10% 3.0 |  
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | GotYourBach  | 3.39  | 1 | GotYourBach  | 12.0 
2 | MissNemisis  | 2.61  | 2 | mypremeclean |  7.0 
3 | mypremeclean | 2.28  | 3 | MissNemisis  |  6.0 
4 | SloshySole   | 1.93  | 4 | SloshySole   |  4.0 
5 | drumcon11    | 0.52  | 5 | drumcon11    | 1.22 


...

Metric: Squad Wipes
Top 1% 2.0 |   Top 5% 1.0 |   Top 10% 1.0 |  
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | mypremeclean | 0.64  | 1 | GotYourBach  |   5  
2 | SloshySole   | 0.59  | 2 | SloshySole   |   3  
3 | GotYourBach  |  0.5  | 3 | mypremeclean |   3  
4 | MissNemisis  | 0.44  | 4 | MissNemisis  |   2  
5 | drumcon11    |  0.0  | 5 | drumcon11    |   0  


Metric: Orders Completed
Top 1% 15.0 |  Top 5% 9.0 |   Top 10% 7.0 |  
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | MissNemisis  | 7.75  | 1 | GotYourBach  |  17  
2 | GotYourBach  | 7.69  | 2 | MissNemisis  |  17  
3 | mypremeclean | 6.86  | 3 | SloshySole   |  17  
4 | SloshySole   | 6.14  | 4 | mypremeclean |  17  
5 | drumcon11    | 3.86  | 5 | drumcon11    |   6  


Metric: Overall Rank
Top 1% 93.0 |  Top 5% 78.0 |  Top 10% 71.0 | 
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | drumcon11    | 36.86 | 1 | mypremeclean |  74  
2 | mypremeclean | 23.41 | 2 | GotYourBach  |  68  
3 | SloshySole   | 11.62 | 3 | SloshySole   |  67  
4 | GotYourBach  | 11.06 | 4 | MissNemisis  |  63  
5 | MissNemisis  | 10.19 | 5 | drumcon11    |  52  


Metric: Team Rank
Top 1% 43.0 |  Top 5% 37.0 |  Top 10% 34.0 | 
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | drumcon11    | 17.86 | 1 | mypremeclean |  44  
2 | mypremeclean | 14.18 | 2 | SloshySole   |  39  
3 | GotYourBach  | 6.31  | 3 | GotYourBach  |  30  
4 | SloshySole   | 6.31  | 4 | MissNemisis  |  28  
5 | MissNemisis  | 5.88  | 5 | drumcon11    |  27  


Metric: Player Time
Top 1% 39.0 |  Top 5% 31.0 |  Top 10% 27.0 | 
Current Leaderboard: 
  Average Performance    |     Peak Performance    
1 | MissNemisis  | 17.06 | 1 | SloshySole   | 37.0 
2 | GotYourBach  | 16.75 | 2 | mypremeclean | 34.0 
3 | mypremeclean | 15.86 | 3 | MissNemisis  | 34.0 
4 | SloshySole   | 15.66 | 4 | GotYourBach  | 34.0 
5 | drumcon11    | 12.57 | 5 | drumcon11    | 18.0 


In [56]:
df.columns

Index(['_id', 'match_id', 'map', 'mode', 'game_duration_m', 'team',
       'team_status', 'player_id', 'kills', 'deaths', 'kills_per_death',
       'kills_per_min', 'solider_damages', 'headshots', 'kill_assists',
       'avenger_kills', 'savior_kills', 'shots_taken', 'shots_hit',
       'shot_accuracy', 'dogtags_taken', 'highest_killstreak',
       'highest_multikill', 'heals', 'revives', 'revives_recieved',
       'resupplies', 'repairs', 'squad_spawns', 'squad_wipes',
       'orders_completed', 'score', 'score_per_min', 'longest_headshot',
       'true_deaths', 'true_kills_per_death', 'overall_rank', 'team_rank',
       'match_start_time', 'player_time', 'AER', 'aggression_rating',
       'efficiency_rating', 'team_orientation', 'week'],
      dtype='object')